# International Collaboration Analysis

In [ ]:
from dimcli.shortcuts import dslquery_json as dslquery
import pandas as pd

## Identify Single institution papers

In [ ]:
sole_institution = pd.DataFrame(
    dslquery("""
        search publications 
            where count(research_orgs) = 1
            and type="article"
            and year = [2007:2017]
        return research_orgs limit 200
        """)['research_orgs']
)
sole_institution.head()

## Identify papers with domestic collaboration

In [ ]:
domestic_instituion = pd.DataFrame(
    dslquery("""
        search publications 
            where count(research_org_countries) = 1
            and type="article"
            and year = [2007:2017]
        return research_orgs limit 200
        """)['research_orgs']
)
domestic_instituion.head()

## Identify papers with international collaboration

In [ ]:
international_instituion = pd.DataFrame(
    dslquery("""
        search publications 
            where count(research_org_countries) > 1
            and type="article"
            and year = [2007:2017]
        return research_orgs limit 200
        """)['research_orgs']
)
international_instituion.head()

## Join the tables together

In [ ]:
sole = sole_institution[['id','name','country_name','count']]
sole.columns = ['id','name','country_name','internal publications']

In [ ]:
dom = domestic_instituion[['id','count']]
dom.columns = ['id','domestic_publications']
international = international_instituion[['id','count']]
international.columns = ['id','international_publications']


In [ ]:
jdf = sole.set_index('id').join(dom.set_index('id')).join(international.set_index('id'))

jdf['internal_international_ratio'] = jdf['internal publications'] / jdf['international_publications']
jdf.head(100)

## Add a little bit of analysis

In [ ]:
def highlight_cols(s):
    if s > 1 :
        color = 'yellow'
    elif s < 1:
        color = 'grey'
    else:
        color = 'white'
    return 'background-color: %s' % color

jdf.style.applymap(highlight_cols, subset=['internal_international_ratio'])